# UMPC-Food101 - Text Classification

In [1]:
from os import path as osp
import os
import logging
from torch.serialization import save

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.modules import activation
from torch.utils.data import DataLoader
from torch import nn,optim
import torchvision.transforms as transforms
from transformers import BertModel, BertConfig,AutoModel
from torch_lr_finder import LRFinder
import random

from TextModels.Text_main import main
from UMPC_Dataset import UMPC_FoodDataset

import random
import nltk
nltk.download('stopwords')

import requests
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
os.environ['CURL_CA_BUNDLE'] = ''


a=15
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
random.seed(a)
np.random.seed(a)
torch.manual_seed(a)
torch.cuda.manual_seed(a)

[nltk_data] Downloading package stopwords to /home1/ece1/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
OUTPUT_SIZE =101
epochs=15
batch_size=128
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

## 1. Train/Test Splits and DataLoaders 

In [3]:
train_set=UMPC_FoodDataset(targ_dir="./datasets/Food101" ,phase='train', mode="Text_only")
train_loader = DataLoader(
    train_set, batch_size=batch_size, shuffle=True, num_workers=0)

test_set=UMPC_FoodDataset(targ_dir="./datasets/Food101" ,phase="test", mode="Text_only")
test_loader = DataLoader(
    test_set, batch_size=batch_size, shuffle=False, num_workers=0)

In [4]:
print ("Train_set: "+str(len(train_set))+",  Test_set: "+str(len(test_set)))
print(f"Total no. of batches in trainloader : {len(train_loader)}")
print(f"Total no. of batches in testloader : {len(test_loader)}")

Train_set: 67988,  Test_set: 22716
Total no. of batches in trainloader : 532
Total no. of batches in testloader : 178


### Samples of dataset : trainset = tuple (txt_tokens,txt,label)

In [5]:
train_set[0]

({'input_ids': tensor([  101, 13675,  7432,  1011,  8962,  6456, 13675,  7432,  1011,  8962,
           6207, 11345, 23377, 14014,   102,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0]),
  'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
  'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])},
 'Crock-Pot Ladies  Crock-Pot Apple Pie Moonshine',
 0)

In [6]:
randomlist = random.sample(range(0, len(train_set)), 4)
for idx in randomlist:
    txt_tokens,txt,label=train_set[idx]
    print (f"Text:{txt}\nClass:{label}-{train_set.idx_to_class[label]}\n\n")

Text:Super Bowl Recipes | Whisk Together
Class:40-french_fries


Text:Persian Baklava Recipe by the.instructor | iFood.tv
Class:2-baklava


Text:Bibimbap (Mixed rice with vegetables) recipe - Maangchi.com
Class:7-bibimbap


Text:Top 10 deviled eggs recipes: From curried to truffled | HellaWella
Class:30-deviled_eggs




## 2. Model

In [7]:
class TextModel(nn.Module):
    def __init__(self, dim_text_repr=768, num_class=101):
        super().__init__()
        config = BertConfig()
        self.textEncoder= BertModel(config).from_pretrained('bert-base-uncased')    
        self.linear = nn.Linear(dim_text_repr, num_class)

    def forward(self, x):
        text = x
        hidden_states = self.textEncoder(**text)  # B, T, dim_text_repr
        e_i = F.dropout(hidden_states[1]) 
        return self.linear(e_i)

In [8]:
model = TextModel(num_class=OUTPUT_SIZE).to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## 3. Loss Function

In [9]:
loss_fn = nn.CrossEntropyLoss()

## 4. Optimizer 

In [9]:
learning_rate = 1e-2
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

## 5.Training and Testing

In [10]:
main(model,train_loader,test_loader,test_loader,loss_fn,
     optimizer,lr=learning_rate ,name="BERT_UMPC",epochs=20,save_weights=True,scheduler_bol=True)

Ep[Train]1/20: 100%|██| 532/532 [03:27<00:00,  2.56it/s, loss=4.081, acc=0.1120]
Epoch[Test]1/20: 100%|█| 178/178 [00:28<00:00,  6.17it/s, loss=2.865, acc=0.4207
Ep[Train]2/20: 100%|██| 532/532 [03:27<00:00,  2.56it/s, loss=1.943, acc=0.6861]
Epoch[Test]2/20: 100%|█| 178/178 [00:28<00:00,  6.16it/s, loss=1.204, acc=0.8156
Ep[Train]3/20: 100%|██| 532/532 [03:28<00:00,  2.56it/s, loss=1.080, acc=0.8261]
Epoch[Test]3/20: 100%|█| 178/178 [00:28<00:00,  6.17it/s, loss=0.888, acc=0.8359
Ep[Train]4/20: 100%|██| 532/532 [03:28<00:00,  2.56it/s, loss=0.883, acc=0.8366]
Epoch[Test]4/20: 100%|█| 178/178 [00:28<00:00,  6.14it/s, loss=0.824, acc=0.8406
Ep[Train]5/20: 100%|██| 532/532 [03:28<00:00,  2.56it/s, loss=0.811, acc=0.8411]
Epoch[Test]5/20: 100%|█| 178/178 [00:28<00:00,  6.16it/s, loss=0.835, acc=0.8379
Ep[Train]6/20: 100%|██| 532/532 [03:28<00:00,  2.56it/s, loss=0.765, acc=0.8466]
Epoch[Test]6/20: 100%|█| 178/178 [00:28<00:00,  6.15it/s, loss=0.783, acc=0.8443
Ep[Train]7/20: 100%|██| 532/

Epoch 00016: reducing learning rate of group 0 to 1.0000e-03.


Ep[Train]17/20: 100%|█| 532/532 [03:28<00:00,  2.55it/s, loss=0.502, acc=0.8849]
Epoch[Test]17/20: 100%|█| 178/178 [00:28<00:00,  6.14it/s, loss=0.741, acc=0.857
Ep[Train]18/20: 100%|█| 532/532 [03:28<00:00,  2.55it/s, loss=0.492, acc=0.8867]
Epoch[Test]18/20: 100%|█| 178/178 [00:28<00:00,  6.15it/s, loss=0.744, acc=0.857
Ep[Train]19/20: 100%|█| 532/532 [03:28<00:00,  2.56it/s, loss=0.487, acc=0.8880]
Epoch[Test]19/20: 100%|█| 178/178 [00:28<00:00,  6.14it/s, loss=0.744, acc=0.857


Epoch 00019: reducing learning rate of group 0 to 1.0000e-04.


Ep[Train]20/20: 100%|█| 532/532 [03:28<00:00,  2.55it/s, loss=0.482, acc=0.8892]
Epoch[Test]20/20: 100%|█| 178/178 [00:28<00:00,  6.16it/s, loss=0.742, acc=0.858
Epoch[Test]1/1: 100%|█| 178/178 [00:28<00:00,  6.17it/s, loss=0.742, acc=0.8580]


## 6.Load Saved weights

In [11]:
model = TextModel(num_class=OUTPUT_SIZE).to(device)
model.load_state_dict(torch.load("./Saved_weights/BERT_UMPC_best_weights.pt"))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<All keys matched successfully>

In [12]:
main(model,train_loader,test_loader,test_loader,loss_fn,optimizer,lr=learning_rate,Test=True)

Epoch[Test]1/1: 100%|█| 178/178 [00:28<00:00,  6.20it/s, loss=0.741, acc=0.8528]
